## บทนำ

บทเรียนนี้จะครอบคลุมถึง:
- ฟังก์ชันคอลคืออะไร และใช้งานในกรณีใดบ้าง
- วิธีสร้างฟังก์ชันคอลโดยใช้ Azure OpenAI
- วิธีผสานฟังก์ชันคอลเข้ากับแอปพลิเคชัน

## เป้าหมายการเรียนรู้

หลังจากจบบทเรียนนี้ คุณจะสามารถและเข้าใจว่า:

- จุดประสงค์ของการใช้ฟังก์ชันคอล
- การตั้งค่าฟังก์ชันคอลโดยใช้ Azure Open AI Service
- ออกแบบฟังก์ชันคอลให้มีประสิทธิภาพสำหรับกรณีการใช้งานของแอปพลิเคชันของคุณ


## ทำความเข้าใจเกี่ยวกับ Function Calls

สำหรับบทเรียนนี้ เราต้องการสร้างฟีเจอร์ให้กับสตาร์ทอัพด้านการศึกษาของเราที่ช่วยให้ผู้ใช้สามารถใช้แชทบอทเพื่อค้นหาคอร์สเทคนิคต่าง ๆ ได้ เราจะช่วยแนะนำคอร์สที่เหมาะกับระดับทักษะ ตำแหน่งงานปัจจุบัน และเทคโนโลยีที่ผู้ใช้สนใจ

เพื่อให้ทำสิ่งนี้ได้ เราจะใช้เครื่องมือผสมผสานกันดังนี้:
 - `Azure Open AI` เพื่อสร้างประสบการณ์แชทให้กับผู้ใช้
 - `Microsoft Learn Catalog API` เพื่อช่วยให้ผู้ใช้ค้นหาคอร์สตามที่ต้องการ
 - `Function Calling` เพื่อรับคำถามของผู้ใช้และส่งไปยังฟังก์ชันเพื่อเรียก API

ก่อนจะเริ่ม มาดูกันว่าทำไมเราถึงควรใช้ function calling ตั้งแต่แรก:

print("Messages in next request:")
print(messages)
print()

second_response = client.chat.completions.create(
    messages=messages,
    model=deployment,
    function_call="auto",
    functions=functions,
    temperature=0
        )  # ขอรับคำตอบใหม่จาก GPT ที่สามารถเห็นผลลัพธ์จากฟังก์ชันได้


print(second_response.choices[0].message)


### ทำไมต้องใช้ Function Calling

ถ้าคุณเคยเรียนบทเรียนอื่นในคอร์สนี้มาก่อน คุณน่าจะเข้าใจถึงพลังของการใช้ Large Language Models (LLMs) แล้ว และก็น่าจะเห็นข้อจำกัดบางอย่างของมันด้วย

Function Calling เป็นฟีเจอร์ของ Azure Open AI Service ที่ถูกออกแบบมาเพื่อแก้ไขข้อจำกัดเหล่านี้:
1) รูปแบบการตอบกลับที่สม่ำเสมอ
2) ความสามารถในการนำข้อมูลจากแหล่งอื่น ๆ ของแอปพลิเคชันมาใช้ในบริบทของแชท

ก่อนที่จะมี function calling การตอบกลับจาก LLM มักจะไม่มีโครงสร้างและไม่สม่ำเสมอ นักพัฒนาต้องเขียนโค้ดตรวจสอบความถูกต้องที่ซับซ้อนเพื่อให้สามารถจัดการกับรูปแบบการตอบกลับที่หลากหลายได้

ผู้ใช้ไม่สามารถถามคำถามอย่างเช่น "ตอนนี้อากาศที่สตอกโฮล์มเป็นอย่างไร?" ได้ เพราะโมเดลถูกจำกัดอยู่แค่ข้อมูลที่ใช้ฝึกเท่านั้น

ลองดูตัวอย่างด้านล่างนี้ที่แสดงให้เห็นถึงปัญหานี้:

สมมติว่าเราต้องการสร้างฐานข้อมูลนักเรียนเพื่อแนะนำคอร์สที่เหมาะสมให้กับพวกเขา ด้านล่างนี้คือลักษณะการบรรยายของนักเรียนสองคนที่มีข้อมูลคล้ายกันมาก


In [ ]:
student_1_description="Emily Johnson is a sophomore majoring in computer science at Duke University. She has a 3.7 GPA. Emily is an active member of the university's Chess Club and Debate Team. She hopes to pursue a career in software engineering after graduating."
 
student_2_description = "Michael Lee is a sophomore majoring in computer science at Stanford University. He has a 3.8 GPA. Michael is known for his programming skills and is an active member of the university's Robotics Club. He hopes to pursue a career in artificial intelligence after finishing his studies."

เราต้องการส่งข้อมูลนี้ไปยัง LLM เพื่อให้มันแยกวิเคราะห์ข้อมูล หลังจากนั้นเราสามารถนำข้อมูลนี้ไปใช้ในแอปพลิเคชันของเราเพื่อส่งไปยัง API หรือเก็บไว้ในฐานข้อมูล

มาสร้าง prompt สองชุดที่เหมือนกัน เพื่อสั่งให้ LLM ทราบว่าข้อมูลแบบไหนที่เราต้องการ:


เราต้องการส่งข้อความนี้ไปยัง LLM เพื่อแยกส่วนที่สำคัญต่อผลิตภัณฑ์ของเรา ดังนั้นเราจึงสามารถสร้างพรอมต์ที่เหมือนกันสองชุดเพื่อสั่งงาน LLM ได้


In [ ]:
prompt1 = f'''
Please extract the following information from the given text and return it as a JSON object:

name
major
school
grades
club

This is the body of text to extract the information from:
{student_1_description}
'''


prompt2 = f'''
Please extract the following information from the given text and return it as a JSON object:

name
major
school
grades
club

This is the body of text to extract the information from:
{student_2_description}
'''


หลังจากสร้างพรอมต์ทั้งสองนี้แล้ว เราจะส่งไปยัง LLM โดยใช้ `openai.ChatCompletion` เราเก็บพรอมต์ไว้ในตัวแปร `messages` และกำหนดบทบาทเป็น `user` ซึ่งเป็นการจำลองข้อความจากผู้ใช้ที่เขียนถึงแชทบอท


In [ ]:
import os
import json
from openai import AzureOpenAI
from dotenv import load_dotenv
load_dotenv()

client = AzureOpenAI(
  api_key=os.environ['AZURE_OPENAI_API_KEY'],  # this is also the default, it can be omitted
  api_version = "2023-07-01-preview"
  )

deployment=os.environ['AZURE_OPENAI_DEPLOYMENT']

: 

In [ ]:
openai_response1 = client.chat.completions.create(
 model=deployment,    
 messages = [{'role': 'user', 'content': prompt1}]
)
openai_response1.choices[0].message.content 

In [ ]:
openai_response2 = client.chat.completions.create(
 model=deployment,    
 messages = [{'role': 'user', 'content': prompt2}]
)
openai_response2.choices[0].message.content

In [ ]:
# Loading the response as a JSON object
json_response1 = json.loads(openai_response1.choices[0].message.content)
json_response1

In [ ]:
# Loading the response as a JSON object
json_response2 = json.loads(openai_response2.choices[0].message.content )
json_response2

ถึงแม้ว่าคำสั่งจะเหมือนกันและคำอธิบายจะคล้ายกัน แต่เราก็อาจได้รูปแบบของ property `Grades` ที่ต่างกัน

ถ้าคุณรันเซลล์ข้างบนหลายครั้ง รูปแบบที่ได้อาจเป็น `3.7` หรือ `3.7 GPA`

สาเหตุก็เพราะ LLM รับข้อมูลที่ไม่มีโครงสร้างในรูปแบบของ prompt ที่เขียนขึ้น และก็คืนค่าข้อมูลที่ไม่มีโครงสร้างเช่นกัน เราจำเป็นต้องมีรูปแบบข้อมูลที่มีโครงสร้าง เพื่อที่เราจะได้รู้ว่าควรคาดหวังข้อมูลแบบไหนเมื่อจะจัดเก็บหรือใช้งานข้อมูลนี้

โดยการใช้ functional calling เราสามารถมั่นใจได้ว่าเราจะได้รับข้อมูลที่มีโครงสร้างกลับมา เวลาที่ใช้ function calling จริง ๆ แล้ว LLM จะไม่ได้เรียกหรือรันฟังก์ชันใด ๆ จริง ๆ แต่เราจะสร้างโครงสร้างให้ LLM ทำตามสำหรับการตอบกลับ จากนั้นเราก็ใช้ข้อมูลที่มีโครงสร้างนี้เพื่อรู้ว่าควรจะรันฟังก์ชันไหนในแอปพลิเคชันของเรา


### กรณีการใช้งานสำหรับการเรียกใช้ฟังก์ชัน

**การเรียกใช้เครื่องมือภายนอก**  
แชทบอทเหมาะกับการให้คำตอบกับคำถามของผู้ใช้มาก เมื่อใช้การเรียกฟังก์ชัน แชทบอทจะสามารถนำข้อความจากผู้ใช้ไปทำงานบางอย่างได้ เช่น นักเรียนอาจขอให้แชทบอท "ส่งอีเมลถึงอาจารย์ของฉัน บอกว่าฉันต้องการความช่วยเหลือเพิ่มเติมในวิชานี้" ซึ่งจะสามารถเรียกใช้ฟังก์ชัน `send_email(to: string, body: string)`

**สร้าง API หรือคำสั่งค้นหาฐานข้อมูล**  
ผู้ใช้สามารถค้นหาข้อมูลด้วยภาษาธรรมชาติที่ถูกแปลงเป็นคำสั่งค้นหาหรือ API ที่มีรูปแบบชัดเจน ตัวอย่างเช่น ครูอาจถามว่า "นักเรียนคนไหนที่ทำงานล่าสุดเสร็จแล้วบ้าง" ซึ่งอาจเรียกใช้ฟังก์ชันชื่อ `get_completed(student_name: string, assignment: int, current_status: string)`

**สร้างข้อมูลที่มีโครงสร้าง**  
ผู้ใช้สามารถนำข้อความหรือไฟล์ CSV มาให้ LLM ช่วยดึงข้อมูลสำคัญออกมาได้ เช่น นักเรียนอาจแปลงบทความวิกิพีเดียเกี่ยวกับข้อตกลงสันติภาพเพื่อสร้างแฟลชการ์ด AI วิธีนี้สามารถทำได้โดยใช้ฟังก์ชันชื่อ `get_important_facts(agreement_name: string, date_signed: string, parties_involved: list)`


## 2. การสร้างการเรียกใช้ฟังก์ชันครั้งแรกของคุณ

ขั้นตอนการสร้างการเรียกใช้ฟังก์ชันประกอบด้วย 3 ขั้นตอนหลัก:
1. เรียกใช้ Chat Completions API พร้อมกับรายการฟังก์ชันของคุณและข้อความจากผู้ใช้
2. อ่านคำตอบของโมเดลเพื่อดำเนินการบางอย่าง เช่น เรียกใช้ฟังก์ชันหรือ API
3. เรียกใช้ Chat Completions API อีกครั้ง พร้อมกับผลลัพธ์ที่ได้จากฟังก์ชันของคุณ เพื่อนำข้อมูลนั้นมาใช้สร้างคำตอบให้กับผู้ใช้


### องค์ประกอบของการเรียกใช้ฟังก์ชัน

#### ข้อมูลที่ผู้ใช้ป้อน

ขั้นตอนแรกคือการสร้างข้อความจากผู้ใช้ ซึ่งสามารถกำหนดค่าได้แบบไดนามิกโดยรับค่าจากกล่องข้อความ หรือจะกำหนดค่าตรงนี้เลยก็ได้ หากนี่เป็นครั้งแรกที่คุณใช้งาน Chat Completions API เราต้องกำหนด `role` และ `content` ของข้อความ

`role` สามารถเป็นได้ทั้ง `system` (สร้างกฎ), `assistant` (โมเดล), หรือ `user` (ผู้ใช้งาน) สำหรับการเรียกใช้ฟังก์ชัน เราจะกำหนดเป็น `user` พร้อมตัวอย่างคำถาม


In [ ]:
messages= [ {"role": "user", "content": "Find me a good course for a beginner student to learn Azure."} ]

### การสร้างฟังก์ชัน

ต่อไปเราจะกำหนดฟังก์ชันและพารามิเตอร์ของฟังก์ชันนั้น ที่นี่เราจะใช้เพียงฟังก์ชันเดียวชื่อว่า `search_courses` แต่คุณสามารถสร้างฟังก์ชันได้หลายอัน

**สำคัญ** : ฟังก์ชันจะถูกรวมอยู่ใน system message ที่ส่งไปยัง LLM และจะถูกนับรวมในจำนวนโทเคนที่คุณมีอยู่ด้วย


In [ ]:
functions = [
   {
      "name":"search_courses",
      "description":"Retrieves courses from the search index based on the parameters provided",
      "parameters":{
         "type":"object",
         "properties":{
            "role":{
               "type":"string",
               "description":"The role of the learner (i.e. developer, data scientist, student, etc.)"
            },
            "product":{
               "type":"string",
               "description":"The product that the lesson is covering (i.e. Azure, Power BI, etc.)"
            },
            "level":{
               "type":"string",
               "description":"The level of experience the learner has prior to taking the course (i.e. beginner, intermediate, advanced)"
            }
         },
         "required":[
            "role"
         ]
      }
   }
]

**คำนิยาม**

`name` - ชื่อของฟังก์ชันที่เราต้องการให้ถูกเรียกใช้งาน

`description` - คำอธิบายเกี่ยวกับวิธีการทำงานของฟังก์ชันนี้ ตรงนี้ควรอธิบายให้ชัดเจนและเฉพาะเจาะจง

`parameters` - รายการของค่าและรูปแบบที่คุณต้องการให้โมเดลสร้างขึ้นในคำตอบ

`type` - ประเภทข้อมูลของ properties ที่จะถูกจัดเก็บไว้

`properties` - รายการของค่าที่เฉพาะเจาะจงที่โมเดลจะใช้สำหรับคำตอบ

`name` - ชื่อของ property ที่โมเดลจะใช้ในคำตอบที่จัดรูปแบบแล้ว

`type` - ประเภทข้อมูลของ property นี้

`description` - คำอธิบายของ property เฉพาะเจาะจงนี้

**ตัวเลือกเพิ่มเติม**

`required` - property ที่จำเป็นสำหรับการเรียกใช้งานฟังก์ชันให้สมบูรณ์


### การเรียกใช้งานฟังก์ชัน
หลังจากที่เราได้กำหนดฟังก์ชันแล้ว ขั้นตอนถัดไปคือการนำฟังก์ชันนั้นไปใช้ในการเรียก Chat Completion API โดยเราจะเพิ่ม `functions` เข้าไปในคำขอ ในที่นี้คือ `functions=functions`

นอกจากนี้ เรายังสามารถตั้งค่า `function_call` ให้เป็น `auto` ได้ด้วย ซึ่งหมายความว่าเราจะให้ LLM เป็นผู้ตัดสินใจเองว่าควรเรียกใช้ฟังก์ชันใดตามข้อความที่ผู้ใช้ส่งมา แทนที่เราจะกำหนดเอง


In [ ]:
response = client.chat.completions.create(model=deployment, 
                                        messages=messages,
                                        functions=functions, 
                                        function_call="auto") 

print(response.choices[0].message)

ตอนนี้เรามาดูที่การตอบกลับและดูว่ามีรูปแบบอย่างไร:

{
  "role": "assistant",
  "function_call": {
    "name": "search_courses",
    "arguments": "{\n  \"role\": \"student\",\n  \"product\": \"Azure\",\n  \"level\": \"beginner\"\n}"
  }
}

คุณจะเห็นว่าชื่อของฟังก์ชันถูกเรียกใช้ และจากข้อความของผู้ใช้ LLM สามารถหาข้อมูลมาใส่ใน arguments ของฟังก์ชันได้


## 3.การผสานการเรียกใช้งานฟังก์ชันเข้ากับแอปพลิเคชัน

หลังจากที่เราได้ทดสอบรูปแบบการตอบกลับจาก LLM แล้ว ตอนนี้เราสามารถนำสิ่งนี้ไปผสานเข้ากับแอปพลิเคชันของเราได้

### การจัดการลำดับขั้นตอน

เพื่อผสานสิ่งนี้เข้ากับแอปพลิเคชันของเรา ให้ทำตามขั้นตอนดังนี้:

ขั้นแรก ให้เราเรียกใช้งานบริการของ Open AI และเก็บข้อความที่ได้ไว้ในตัวแปรชื่อ `response_message`


In [ ]:
response_message = response.choices[0].message

ตอนนี้เราจะกำหนดฟังก์ชันที่จะเรียก Microsoft Learn API เพื่อดึงรายการคอร์ส:


In [ ]:
import requests

def search_courses(role, product, level):
    url = "https://learn.microsoft.com/api/catalog/"
    params = {
        "role": role,
        "product": product,
        "level": level
    }
    response = requests.get(url, params=params)
    modules = response.json()["modules"]
    results = []
    for module in modules[:5]:
        title = module["title"]
        url = module["url"]
        results.append({"title": title, "url": url})
    return str(results)



เป็นแนวทางปฏิบัติที่ดี เราจะตรวจสอบก่อนว่าระบบต้องการเรียกฟังก์ชันหรือไม่ หลังจากนั้น เราจะสร้างหนึ่งในฟังก์ชันที่มีอยู่และจับคู่กับฟังก์ชันที่ถูกเรียก

จากนั้น เราจะนำอาร์กิวเมนต์ของฟังก์ชันนั้นมาแมปกับอาร์กิวเมนต์ที่ได้จาก LLM

สุดท้าย เราจะเพิ่มข้อความเรียกฟังก์ชันและค่าที่ได้จากข้อความ `search_courses` เข้าไปด้วย วิธีนี้จะทำให้ LLM มีข้อมูลครบถ้วนสำหรับตอบกลับผู้ใช้เป็นภาษาธรรมชาติ


In [ ]:
# Check if the model wants to call a function
if response_message.function_call.name:
    print("Recommended Function call:")
    print(response_message.function_call.name)
    print()

    # Call the function. 
    function_name = response_message.function_call.name

    available_functions = {
            "search_courses": search_courses,
    }
    function_to_call = available_functions[function_name] 

    function_args = json.loads(response_message.function_call.arguments)
    function_response = function_to_call(**function_args)

    print("Output of function call:")
    print(function_response)
    print(type(function_response))


    # Add the assistant response and function response to the messages
    messages.append( # adding assistant response to messages
        {
            "role": response_message.role,
            "function_call": {
                "name": function_name,
                "arguments": response_message.function_call.arguments,
            },
            "content": None
        }
    )
    messages.append( # adding function response to messages
        {
            "role": "function",
            "name": function_name,
            "content":function_response,
        }
    )



In [ ]:
print("Messages in next request:")
print(messages)
print()

second_response = client.chat.completions.create(
    messages=messages,
    model=deployment,
    function_call="auto",
    functions=functions,
    temperature=0
        )  # get a new response from GPT where it can see the function response


print(second_response.choices[0].message)

## โจทย์ท้าทายโค้ด

เยี่ยมมาก! ถ้าอยากเรียนรู้เพิ่มเติมเกี่ยวกับ Azure Open AI Function Calling คุณสามารถลองสร้างตามนี้: https://learn.microsoft.com/training/support/catalog-api-developer-reference?WT.mc_id=academic-105485-koreyst
 - เพิ่มพารามิเตอร์ของฟังก์ชันให้มากขึ้น เพื่อช่วยให้ผู้เรียนค้นหาคอร์สได้ง่ายขึ้น คุณสามารถดูพารามิเตอร์ของ API ที่มีได้ที่นี่:
 - สร้างฟังก์ชันอีกตัวที่รับข้อมูลเพิ่มเติมจากผู้เรียน เช่น ภาษาแม่ของผู้เรียน
 - สร้างการจัดการข้อผิดพลาด เมื่อการเรียกฟังก์ชันหรือ API ไม่สามารถคืนคอร์สที่เหมาะสมได้



---

**ข้อจำกัดความรับผิดชอบ**:  
เอกสารฉบับนี้ได้รับการแปลโดยใช้บริการแปลภาษา AI [Co-op Translator](https://github.com/Azure/co-op-translator) แม้ว่าเราจะพยายามอย่างเต็มที่เพื่อความถูกต้อง แต่โปรดทราบว่าการแปลโดยระบบอัตโนมัติอาจมีข้อผิดพลาดหรือความไม่ถูกต้อง เอกสารต้นฉบับในภาษาต้นทางควรถือเป็นแหล่งข้อมูลที่เชื่อถือได้ สำหรับข้อมูลที่มีความสำคัญ แนะนำให้ใช้บริการแปลโดยนักแปลมืออาชีพ ทางเราจะไม่รับผิดชอบต่อความเข้าใจผิดหรือการตีความที่คลาดเคลื่อนซึ่งเกิดจากการใช้การแปลนี้
